In [3]:
!pip install onepanel-sdk

In [1]:
import os
import sys
from time import sleep

import onepanel.core.api
from onepanel.core.api.rest import ApiException
from onepanel.core.api.models import Parameter

In [8]:
# If outside of Onepanel
# token = '<auth-token>'
# Set environment variables for:
#    ONEPANEL_API_URL, example: https://app.domain.com/api
#    ONEPANEL_RESOURCE_NAMESPACE, example: default

# If inside of Onepanel, get mounted service account token to use as API Key
with open('/var/run/secrets/kubernetes.io/serviceaccount/token') as f:
    token = f.read()

# Configure API key authorization: Bearer
configuration = onepanel.core.api.Configuration(
    host=os.getenv('ONEPANEL_API_URL'),
    api_key={
        'authorization': token
    }
)
configuration.api_key_prefix['authorization'] = 'Bearer'

# Enter a context with an instance of the API client
with onepanel.core.api.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = onepanel.core.api.WorkspaceServiceApi(api_client)

    namespace = os.getenv('ONEPANEL_RESOURCE_NAMESPACE')
    params = []
    params.append(Parameter(name='sys-name', value='JupyterLab Workspace'))
    params.append(Parameter(name='sys-node-pool', value='Standard_NC6'))
    body = onepanel.core.api.CreateWorkspaceBody(
        parameters=params,
        workspace_template_uid='jupyterlab',
    )
    try:
        workspace = api_instance.create_workspace(namespace, body)
        uid = workspace.uid
        print('Waiting for workspace to launch...\n')
        while workspace.status.phase != 'Running':
            workspace = api_instance.get_workspace(namespace, uid)
            sleep(15)
        print('Workspace is running at %s.\n' % workspace.url)
    except ApiException as e:
        print("Exception when calling WorkspaceServiceApi->create_workspace: %s\n" % e)

Waiting for workspace to launch...

Workspace is running at http://jupyterlab-workspace--rush.rush-gcs.onepanel.site.

